In [1]:
import re
import time
import pickle
import random
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import chromedriver_binary
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException        



In [2]:
main_page_url= "https://www.malabargoldanddiamonds.com/ae/gifts.html?dir=desc&gender=617_618&order=new&product_type=888_889_890&utm_campaign=women-gift-ring&utm_medium=shop-by-category&utm_source=Homepage&utm_term=march-19"

In [3]:
def check_exists_by_xpath(xpath, driver):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException or AttributeError:
        return ""
    return driver.find_element_by_xpath(xpath).text

In [4]:
def check_exists_by_tag(tag, attributes, soup):
    try:
        soup.find(tag, attrs= attributes).text
    except AttributeError:
        return ""
    return soup.find(tag, attrs= attributes).text

In [5]:
def get_page_driver(products_page_url):
    driver = webdriver.Chrome()
    driver.get(products_page_url)
    return driver

In [6]:
def load_full_page(driver):
    men_checkbox= driver.find_element_by_xpath('//*[@id="narrow-by-list"]/div[2]/div[2]/ol/li[2]/label')
    men_checkbox.click()
    counter = 0
    while counter < 10:
        if check_exists_by_xpath('//*[@id="m-show-more"]', driver):
            more_btn = driver.find_element_by_xpath('//*[@id="m-show-more"]')
            more_btn.click()
            counter = 0 
        element = driver.find_element_by_xpath('//*[@id="main-container"]/div/div/div[6]/div[2]/div')
        element.location_once_scrolled_into_view
        counter += 1
        time.sleep(random.randrange(6,10))
    return driver

In [7]:
#takes products page url and returns a list of urls for all of its products
def get_urls(driver):   
    rings_soup=BeautifulSoup(driver.page_source, 'lxml')
    rings_list = rings_soup.find_all('a', attrs = {'class':'product-image'})
    rings_urls = [product['href'] for product in rings_list]
    return rings_urls


In [8]:
# def get_products_details(dataset, products_urls_list):
#     for product_url in products_urls_list:
#         driver = get_page_driver(product_url)
#         soup=BeautifulSoup(driver.page_source, 'lxml') 
#         dataset.loc[len(dataset)] = [driver.find_element_by_xpath('//*[@id="leftparentdiv"]/div[1]/ul/li[1]/span[2]').text, 
#                             check_exists_by_xpath('//*[@id="leftparentdiv"]/div[1]/ul/li[2]/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="leftparentdiv"]/div[1]/ul/li[3]/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="leftparentdiv"]/div[1]/ul/li[4]/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="leftparentdiv"]/div[2]/ul/li[1]/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="leftparentdiv"]/div[2]/ul/li[2]/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="leftparentdiv"]/div[2]/ul/li[3]/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="leftparentdiv"]/div[2]/ul/li[4]/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="leftparentdiv"]/div[3]/ul/li/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="leftparentdiv"]/div[3]/ul/li/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="rightparentdiv"]/div[1]/ul/li/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="rightparentdiv"]/div[2]/ul/li[1]/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="rightparentdiv"]/div[2]/ul/li[2]/span[2]', driver),
#                             check_exists_by_xpath('//*[@id="rightparentdiv"]/div[2]/ul/li[3]/span[2]',driver),
#                             soup.find('span',attrs={'class':'price'}).text]
#         driver.close()
#     return dataset

In [9]:
def get_products_details(dataset, products_urls_list):
    for product_url in products_urls_list:
        response = requests.get(product_url)
        page = response.text
        soup=BeautifulSoup(page, 'lxml') 
        dataset.loc[len(dataset)] = [
                            check_exists_by_tag('span', {'class':'f-right product_type'},soup), 
                            check_exists_by_tag('span', {'class':'f-right brand'},soup),
                            check_exists_by_tag('span', {'class':'f-right item_package_quantity'},soup),
                            check_exists_by_tag('span', {'class':'f-right gender'},soup),
                            check_exists_by_tag('span', {'class':'f-right gold_purity'},soup),
                            check_exists_by_tag('span', {'class':'f-right gold_colour'},soup),
                            check_exists_by_tag('span', {'class':'f-right gross_gold_weight'},soup),
                            check_exists_by_tag('span', {'class':'f-right net_gold_weight'},soup),
                            check_exists_by_tag('span', {'class':'f-right gold_certification'},soup),
                            check_exists_by_tag('span', {'class':'f-right diamond_certification'},soup),
                            check_exists_by_tag('span', {'class':'f-right ring_size'},soup),
                            check_exists_by_tag('span', {'class':'f-right diamond_colour'},soup),
                            check_exists_by_tag('span', {'class':'f-right diamond_carat'},soup),
                            check_exists_by_tag('span', {'class':'f-right diamond_pcs'},soup),
                            check_exists_by_tag('span',{'class':'price'}, soup)]
        print(len(dataset))
    return dataset

In [10]:
# driver = get_page_driver(main_page_url)

In [11]:
# loaded_full_page = load_full_page(driver)

In [12]:
# products_urls = get_urls(loaded_full_page)

In [13]:
# pickle_out = open("products_urls.pickle","wb")
# pickle.dump(products_urls, pickle_out)
# pickle_out.close()

In [14]:
# pickle_in = open("products_urls.pickle","rb")
# products_urls = pickle.load(pickle_in)

In [15]:
# products_urls[275]

In [16]:
# dataset = pd.DataFrame(columns=['product_type','brand','item_package_quantity','gender','purity_KT','metal_color','gross_wieght','net_wieght','g_cert','d_cert','ring_size', 'diamond_color', 'carat_CT', 'diamond_pcs','price'])
#

In [17]:
# pickle_out = open("products_df_bs.pickle","wb")
# pickle.dump(products_df, pickle_out)
# pickle_out.close()

In [18]:
# products_df = get_products_details(dataset, products_urls)

In [20]:
pickle_in = open("products_df_bs.pickle","rb")
products_df = pickle.load(pickle_in)

In [21]:
products_df

,product_type,brand,item_package_quantity,gender,purity_KT,metal_color,gross_wieght,net_wieght,g_cert,d_cert,ring_size,diamond_color,carat_CT,diamond_pcs,price
0,Earring,Malabar,2,Women,22 KT (916),Yellow,7.28,7.28,916,,,,,,USD 435 (Approx)
1,Earring,Malabar,2,Women,22 KT (916),Yellow,4.16,4.16,916,,,,,,USD 243 (Approx)
2,Earring,Malabar,2,Women,22 KT (916),Yellow,5.33,5.33,916,,,,,,USD 326 (Approx)
3,Pendant,Malabar,1,Men,22 KT (916),Yellow,8.02,8.02,916,,,,,,USD 469 (Approx)
4,Pendant,Malabar,1,Women,22 KT (916),Two Tone,2.66,2.66,916,,,,,,USD 162 (Approx)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,Pendant,Malabar,,Women,22 KT (916),Yellow,2.42,2.42,,,,,,,USD 164 (Approx)
1087,Pendant,Malabar,,Women,22 KT (916),Yellow,2.35,2.35,,,,,,,USD 150 (Approx)
1088,Pendant,Malabar,,Women,22 KT (916),Two tone,2.84,2.84,,,,,,,USD 178 (Approx)
1089,Pendant,Malabar,,Women,22 KT (916),Yellow,1.29,1.29,,,,,,,USD 91 (Approx)
